In [ ]:
# imports
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# my project in Google Cloud used
PROJECT = "questrom"

# some settings for pandas
pd.options.display.max_rows=125
pd.options.display.max_columns=100


In [ ]:
# auth against your BU GCP account 
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
# get the data from Big Query
# select all the records  
SQL = """
SELECT *
FROM `questrom.datasets.crm_campaign`
"""

crm = pd.io.gbq.read_gbq(SQL, PROJECT)

In [ ]:
# what do we have
crm.shape

In [ ]:
crm.head()

In [ ]:
#### Exercise 1

In [ ]:
# first, I am going to use granular codes, not the category
# so drop contcat
crm.drop(columns="contcat", inplace=True)


In [ ]:
# second, drop dupes if any
crm.drop_duplicates(inplace=True)

In [ ]:
# last confirm we have missing data
crm.isna().sum()

In [ ]:
# we will remove these records
# even robust client data warehouses will have dirty data!
crm.dropna(inplace=True)

crm.isna().sum()

In [ ]:
# ok, reshape
crm['flag'] = True

db = crm.pivot(index="crm_id", columns="contcode", values="flag")
db.fillna(False, inplace=True)

db.shape

In [ ]:
#### Exercise 2

In [ ]:
# a quick glimpse
db.head(3)

In [ ]:
# lets review how many "converted" via the CMO marketing code
converted = db.CMO.sum() 
print(converted)
db.CMO.mean()

In [ ]:
#2 ten most frequent interactions
interactions = db.sum(axis=0)
interactions.sort_values(ascending=False)[:10]

In [ ]:
#3 avg number of transcations
user_ints = db.sum(axis=1)
user_ints.mean() 

In [ ]:
#4 item frequency
interactions_f = interactions / len(db)

interactions_f.sort_values(ascending=False, inplace=True)

sns.lineplot(range(len(interactions_f)), interactions_f.values)

In [ ]:
#### Exercise 3

In [ ]:
itemsets = apriori(db, min_support=.0002, use_colnames=True)

rules = association_rules(itemsets, metric="support", min_threshold=.0002)

In [ ]:
#### Exercise 4

In [ ]:
# a quick review across the outputs
rules.describe()

In [ ]:
# sample 10 rules
rules.sample(10).head()

In [ ]:
# top 10 lift
rules.sort_values("lift", ascending=False).head(10)

In [ ]:
# top 10 on count
# we can work backwards
rules['count'] = rules.support * len(db)

rules.sort_values('count', ascending=False).head(10)

In [ ]:
# keep just those with 6 on the LHS
# rules that always hold have infinite conviction 
# https://michael.hahsler.net/research/association_rules/measures.html#conviction
rules['lhs_len'] = rules.antecedents.apply(lambda x: len(x))

rules.loc[rules.lhs_len == 6, :].shape

In [ ]:
rules.loc[rules.lhs_len == 6, :].sample(5)

In [ ]:
#### Exercise 5

In [ ]:
# make a copy
strategy = rules.copy()

# calc rhs length
strategy['rhs_len'] = strategy.consequents.apply(lambda x: len(x))

In [ ]:
# keep jus the rules with the sales conversion on the right
ROWS = np.where((strategy.consequents=={'CMO'}) & (strategy.rhs_len == 1))

strategy = strategy.iloc[ROWS[0], :]

In [ ]:
# confirm this is what we intended
strategy.shape

strategy.head()

In [ ]:
# explore -- long sales pipelines == many interactions on the LHS

strategy.sort_values('lhs_len', ascending=False).head(10)

In [ ]:
# some plots

sns.scatterplot(data=strategy, x="support", y="confidence", hue="lift")

In [ ]:
# number of lhs items balanced against support
sns.boxplot(data=strategy, x="lhs_len", y="confidence", color="grey")

## what does this suggest about a customers interactions WHEN, we know that they converted?

In [ ]:
# fastest path
rule1 = strategy.loc[strategy.lhs_len == 1, :]
rule1.sort_values("lift", ascending=False, inplace=True)

rule1.head(10)

In [ ]:
# above, you might recommend flagging those that only have TSA interaction
# or to provide options, by say, filtering on sample size > 500?